In [ ]:
from scipy.stats import ttest_ind
import helper 
import xarray as xr 
import numpy as np 
import cmaps as cmap 
import matplotlib.pyplot as plt
import scipy 
import warnings
import matplotlib.patheffects as pe
from matplotlib.colors import Normalize
from matplotlib import rcParams
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle
import matplotlib.patheffects as pe
warnings.filterwarnings("ignore")

def rePoPolar(dataset, name, offset):
    x = dataset.lon.values / 4
    y = dataset.lat.values / 4
    x, y = np.meshgrid(x, y)

    R = 6371.0
    cphi = np.cos(np.deg2rad(y))
    x = R * np.deg2rad(x) * cphi
    y = R * np.deg2rad(y)

    r = np.sqrt(x**2 + y**2)
    t = np.arctan2(y, x)

    rBins = np.linspace(np.nanmin(r), np.nanmax(r), 120)
    tBins = np.linspace(np.nanmin(t), np.nanmax(t), 360)

    for i in range(len(tBins)):
            tBins[i] = tBins[i] + offset
            while tBins[i] <= (-1 * np.pi):
                tBins[i] = tBins[i] + (2 * np.pi)
            while tBins[i] >= np.pi:
                tBins[i] = tBins[i] - (2 * np.pi)

    R, T = np.meshgrid(rBins, tBins)
    newX, newY = R * np.cos(T), R * np.sin(T)
    gridded_data = scipy.interpolate.griddata((x.flatten(), y.flatten()), dataset.values.flatten(), (newX.flatten(), newY.flatten()), method='nearest')

    polar = xr.Dataset(
        {
            name: (('r', 'theta'), gridded_data.reshape(R.shape).transpose())
        },
        coords={
            'r': rBins,
            'theta': tBins
        }
    )

    return polar


In [ ]:
dataset = xr.open_dataset(r"C:\Users\deela\Downloads\TCRADAR_ERA5.nc")
print(dataset)
print(list(dataset.variables))

In [ ]:
def process(data, ddir):
    dataList = []
    print(len(data.case))
    for x in range(len(data.case)):
        # print(x)
        offset = np.deg2rad(360 - ddir.isel(case = x))
        try:
            temp = rePoPolar(data.isel(case = x), 'rh', offset = offset)
        except:
            temp = xr.Dataset(
                {
                    var: (temp[var].dims, np.full_like(temp[var], np.nan))
                    for var in temp.data_vars
                },
                coords=temp.coords,
                attrs=temp.attrs)

        dataList.append(temp)

    newTheta = np.linspace(-np.pi, np.pi, 360, endpoint=False)
    for x in range(len(dataList)):
        try:
            dataList[x] = dataList[x].interp(theta = newTheta)    
            # dataList[x]['rh'].values = np.flip(dataList[x]['rh'].values, axis = 1)
        except:
            pass

    rh = xr.concat(dataList, dim = 'case')
    print(rh)  

    return rh, newTheta 

In [ ]:
def labels(ax, flag = False):
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    if flag == False:
        ax.text(1 * np.pi / 4, 1000, 'Downshear\nRight', size = 12, color = 'black', horizontalalignment = 'center', fontfamily = 'Courier New', fontweight = 'bold', path_effects=[pe.withStroke(linewidth=2.25, foreground="white")], verticalalignment = 'center')
        ax.text(3 * np.pi / 4, 1000, 'Upshear\nRight', size = 12, color = 'black', horizontalalignment = 'center', fontfamily = 'Courier New', fontweight = 'bold', path_effects=[pe.withStroke(linewidth=2.25, foreground="white")], verticalalignment = 'center')
        ax.text(5 * np.pi / 4, 1000, 'Upshear\nLeft', size = 12, color = 'black', horizontalalignment = 'center', fontfamily = 'Courier New', fontweight = 'bold', path_effects=[pe.withStroke(linewidth=2.25, foreground="white")], verticalalignment = 'center')
        ax.text(7 * np.pi / 4, 1000, 'Downshear\nLeft', size = 12, color = 'black', horizontalalignment = 'center', fontfamily = 'Courier New', fontweight = 'bold', path_effects=[pe.withStroke(linewidth=2.25, foreground="white")], verticalalignment = 'center')
        
        ax.annotate('', xy=(0, 500), xytext=(np.pi, 500),
                arrowprops=dict(facecolor='black', edgecolor='black', width=1, headwidth=8, headlength=10, path_effects=[pe.withStroke(linewidth=2.25, foreground="white")]))

    ax.set_yticklabels(['', '200km', '', '600km', '', '1000km', '', ''], fontfamily = 'Courier New', path_effects=[pe.withStroke(linewidth=2.25, foreground="white")])
    ax.set_xticklabels(['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'], fontfamily = 'Courier New', path_effects=[pe.withStroke(linewidth=2.25, foreground="white")])


In [ ]:
alignment = [225, 251, 252, 253, 254, 333, 334, 347, 374, 376, 377, 407, 408, 409, 410, 413, 414, 603, 604, 605, 672, 712, 719, 752, 765, 864, 878, 879, 939, 941, 957, 968, 969, 970, 971, 1057, 1073, 1101, 1128, 1131, 1148, 1177, 1178, 1179, 1180, 1191, 1192, 1220, 1221, 1222, 1223, 1224, 1226, 1227, 1228, 1302, 1379, 1380, 1391, 1405, 1406, 1445, 1446, 1447, 1448, 1453, 1471]
misalign = [148, 149, 223, 224, 339, 340, 341, 342, 343, 344, 382, 383, 384, 386, 400, 402, 423, 424, 425, 426, 427, 429, 430, 431, 545, 600, 601, 742, 744, 745, 747, 757, 760, 869, 898, 899, 918, 919, 930, 934, 935, 936, 1042, 1052, 1174, 1175, 1195, 1197, 1201, 1217, 1218, 1372, 1373, 1376, 1377, 1408, 1410, 1418, 1419]

In [ ]:
hgt = 200
var = 'Theta-E'
varShort = 'rh'
height = f'{hgt}mb'


In [ ]:
uData = dataset['u_data'] - dataset['uspd']
vData = dataset['v_data'] - dataset['vspd']
ddir = dataset['shear_dir']

In [ ]:
ud = uData.sel(level = hgt)
u, newTheta = process(ud, ddir)
vd = vData.sel(level = hgt)
v, newTheta = process(vd, ddir)

In [ ]:
R, T = np.meshgrid(u.r, newTheta, indexing = 'ij') 
radialWind = (u['rh'] * np.cos(T)) + (v['rh'] * np.sin(T))
radialWind.values = np.flip(radialWind.values, axis = 2)

In [ ]:
print(radialWind)

In [ ]:
radPer = radialWind - radialWind.mean('theta')

In [ ]:
data = helper.thetae(dataset['temperature'], dataset['level'], 1000, dataset['sphum'])
data = data.sel(level = hgt)

ddir = dataset['shear_dir']
thetaE, newTheta = process(data, ddir)

In [ ]:
thetaE['rh'].values = np.flip(thetaE['rh'].values, axis = 2)
thtPer = thetaE['rh'] - thetaE['rh'].mean('theta')

In [ ]:
vent = radPer * thtPer
t = f'{height} Ventilation'

In [ ]:
t = f'{height} Ventilation'

refl1 = vent.sel(case = alignment, r = slice(0, 1500)).mean('case')
refl2 = vent.sel(case = misalign, r = slice(0, 1500)).mean('case')
refl = vent.sel(r = slice(0, 1500)).mean('case')
data = refl1 - refl2
tstat, pval = ttest_ind(vent.sel(case = alignment, r = slice(0, 1500)), vent.sel(case = misalign, r = slice(0, 1500)), axis = 0, equal_var=False, nan_policy='omit')
sig_mask = pval < 0.05

fig, axes = plt.subplots(2, 2, subplot_kw={'projection': 'polar'}, figsize=(18, 15))
axes[0, 0].pcolormesh(newTheta, refl1.r, refl1, cmap = cmap.tempAnoms3(), vmin = -10, vmax = 10)
labels(axes[0, 0])

c = axes[0, 1].pcolormesh(newTheta, refl2.r, refl2, cmap = cmap.tempAnoms3(), vmin = -10, vmax = 10)
labels(axes[0, 1])

ca = axes[1, 0].pcolormesh(newTheta, refl2.r, data, cmap = cmap.tempAnoms3(), vmin = -10, vmax = 10)
axes[1, 0].contour(newTheta, refl2.r, sig_mask, levels=[0.5], colors='black', linewidths=1)
labels(axes[1, 0])

c = axes[1, 1].pcolormesh(newTheta, refl.r, refl, cmap = cmap.tempAnoms3(), vmin = -10, vmax = 10)
labels(axes[1, 1])

cbar = fig.colorbar(c, ax=axes, orientation='vertical', aspect=50, pad=0.04)
cbar.ax.tick_params(axis='both', labelsize=9, left = False, bottom = False)

pos = axes[1, 0].get_position()

cbar_ax = fig.add_axes([
    pos.x0,            
    pos.y0 - 0.025,     
    pos.width,         
    0.005               
])

cbar = fig.colorbar(ca, cax=cbar_ax, orientation='horizontal')

fig.suptitle(f'TC-RADAR: Shear-Relative Composite {t}                               \n\n' , fontweight='bold', fontsize=10, y = 0.93)

axes[0, 0].set_title(f'Aligning (Mean Ventilation < 500km: {str(round(np.nanmean(refl1.sel(r = slice(0, 500))), 1))})', fontsize=9)
axes[0, 1].set_title(f'Non-Aligning (Mean Ventilation < 500km: {str(round(np.nanmean(refl2.sel(r = slice(0, 500))), 1))})', fontsize=9)
axes[1, 0].set_title(f'Difference (Mean Ventilation < 500km: {str(round(np.nanmean(data.sel(r = slice(0, 500))), 1))})', fontsize=9)
axes[1, 1].set_title(f'All Cases (Mean Ventilation < 500km: {str(round(np.nanmean(refl.sel(r = slice(0, 500))), 1))})', fontsize=9)

plt.savefig(r"C:\Users\deela\Downloads\tilt" + t + ".png", dpi = 400, bbox_inches = 'tight')
plt.show()

In [ ]:
t = f'{height} Radial Wind'

refl1 = radialWind.sel(case = alignment, r = slice(0, 1500)).mean('case')
refl2 = radialWind.sel(case = misalign, r = slice(0, 1500)).mean('case')
refl = radialWind.sel(r = slice(0, 1500)).mean('case')
data = refl1 - refl2
# tstat, pval = ttest_ind(radialWind.sel(case = alignment, r = slice(0, 1500)), radialWind.sel(case = misalign, r = slice(0, 1500)), axis = 0, equal_var=False, nan_policy='omit')
# sig_mask = pval < 0.05

fig, axes = plt.subplots(2, 2, subplot_kw={'projection': 'polar'}, figsize=(18, 15))
axes[0, 0].pcolormesh(newTheta, refl1.r, refl1, cmap = cmap.tempAnoms3(), vmin = -10, vmax = 10)
labels(axes[0, 0])

c = axes[0, 1].pcolormesh(newTheta, refl2.r, refl2, cmap = cmap.tempAnoms3(), vmin = -10, vmax = 10)
labels(axes[0, 1])

ca = axes[1, 0].pcolormesh(newTheta, refl2.r, data, cmap = cmap.tempAnoms3(), vmin = -10, vmax = 10)
# axes[1, 0].contour(newTheta, refl2.r, sig_mask, levels=[0.5], colors='black', linewidths=1)
labels(axes[1, 0])

c = axes[1, 1].pcolormesh(newTheta, refl.r, refl, cmap = cmap.tempAnoms3(), vmin = -10, vmax = 10)
labels(axes[1, 1])

cbar = fig.colorbar(c, ax=axes, orientation='vertical', aspect=50, pad=0.04)
cbar.ax.tick_params(axis='both', labelsize=9, left = False, bottom = False)

pos = axes[1, 0].get_position()

cbar_ax = fig.add_axes([
    pos.x0,            
    pos.y0 - 0.025,     
    pos.width,         
    0.005               
])

cbar = fig.colorbar(ca, cax=cbar_ax, orientation='horizontal')

fig.suptitle(f'TC-RADAR: Shear-Relative Composite {t}                               \n\n' , fontweight='bold', fontsize=10, y = 0.93)

axes[0, 0].set_title(f'Aligning (Mean Radial Wind < 500km: {str(round(np.nanmean(refl1.sel(r = slice(0, 500))), 1))})', fontsize=9)
axes[0, 1].set_title(f'Non-Aligning (Mean Radial Wind < 500km: {str(round(np.nanmean(refl2.sel(r = slice(0, 500))), 1))})', fontsize=9)
axes[1, 0].set_title(f'Difference (Mean Radial Wind < 500km: {str(round(np.nanmean(data.sel(r = slice(0, 500))), 1))})', fontsize=9)
axes[1, 1].set_title(f'All Cases (Mean Radial Wind < 500km: {str(round(np.nanmean(refl.sel(r = slice(0, 500))), 1))})', fontsize=9)

plt.savefig(r"C:\Users\deela\Downloads\tilt" + t + ".png", dpi = 400, bbox_inches = 'tight')
plt.show()